# xgBoost

In [5]:
import pandas as pd
import numpy as np 
import os 

CLEAN_DATA_DIR = "../data/clean/"

In [7]:
data = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TRAIN_MERGED_FILLED.csv"))
data.head()

,psu_hh_idcode,hhid,rating_filled,q02,q03,q05,q09,q23,Q01,Q03,Q06,Q07,Q08,Q11,Q19
0,30_8_1,3008,4.0,1,1,44,0,0,1.0,1.0,2.0,1.0,2.0,13.0,2.0
1,194_1_2,19401,1.0,2,2,48,0,0,1.0,1.0,2.0,0.0,2.0,13.0,2.0
2,224_6_1,22406,3.0,1,1,61,0,0,1.0,1.0,2.0,0.0,2.0,13.0,2.0
3,323_10_1,32310,5.0,1,1,66,0,0,1.0,1.0,2.0,0.0,2.0,13.0,2.0
4,428_10_1,42810,4.0,2,1,72,0,0,1.0,1.0,1.0,0.0,2.0,14.0,2.0


In [21]:
from sklearn.model_selection import train_test_split, GridSearchCV

data = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TRAIN_MERGED_FILLED.csv"))
X_train, _, y_train, _ = train_test_split(data.drop(['psu_hh_idcode', 'hhid', 'rating_filled'], axis='columns'), data['rating_filled'], test_size=0.2, random_state=101)

unfilled_data = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TRAIN_MERGED_UNFILLED.csv"))
X_test = unfilled_data.drop(['psu_hh_idcode', 'hhid', 'subjectivePoverty_rating'], axis='columns')
y_test = unfilled_data['subjectivePoverty_rating']

# for some reason xgb requires the labels to begin at 0 instead of 1 so we will shift down by 1
y_train = y_train - 1
y_test = y_test - 1

import xgboost as xgb

xgb_model = xgb.XGBClassifier(objective='multiclass:softprob', eval_metric='mlogloss', learning_rate=0.1, max_depth=1)
xgb_model.fit(X_train, y_train)

preds = xgb_model.predict(X_test)
preds_proba = xgb_model.predict_proba(X_test)


from sklearn.metrics import log_loss
print(f"Log Loss from test: {log_loss(y_test, preds_proba)}")


Log Loss from test: 1.9346025316905149


In [29]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [1, 3, 5],
    'n_estimators': [50, 100, 200],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Create the XGBoost model
xgb_model = xgb.XGBClassifier(objective='multi:softprob', eval_metric='mlogloss', random_state=101)

# Create the GridSearchCV object
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_log_loss',  # Use log loss as the evaluation metric
    cv=3,                    # 3-fold cross-validation
    verbose=2,               # Print progress
    n_jobs=-1                # Use all available cores
)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best log loss score
print("Best Parameters:", grid_search.best_params_)
print("Best Log Loss Score:", -grid_search.best_score_)

# Use the best model to make predictions
best_model = grid_search.best_estimator_
preds_proba = best_model.predict_proba(X_test)

# Evaluate log loss on the test data
from sklearn.metrics import log_loss
print(f"Log Loss from test: {log_loss(y_test, preds_proba)}")


Fitting 3 folds for each of 243 candidates, totalling 729 fits


Baseline log loss = $-log(\frac{1}{\#classes}) = -log(\frac{1}{10}) = 2.303$

In [28]:
print("unique values in test classes:", sorted(list(y_test.unique())))
print("unique values in predicted classes:", sorted(list(np.unique(preds))))

unique values in test classes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
unique values in predicted classes: [2, 3, 4, 5]
